In [ ]:
#recycle RNN2struct for the purpose of GO pred. Use distmat data w aln data from pdb70
#use cath mapping for GO terms
#2 outputs. train on cath pt sequence and distmat pca

import numpy as np
from keras.models import *
from keras.optimizers import *
from keras.layers import *
from keras.callbacks import *
from keras.models import load_model
import datetime
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.preprocessing import StandardScaler,  Normalizer , MinMaxScaler , RobustScaler
from sklearn.decomposition import PCA

epsilon = K.epsilon()
from io import BytesIO, StringIO
from tensorflow.python.lib.io import file_io
import argparse
import keras
global max_epochs

flag_show_plots = True # True for Notebooks, False otherwise
if flag_show_plots:
    import matplotlib.pyplot as plt
    from matplotlib.pyplot import figure

from keras.models import Model
from keras.layers import Flatten, Input, Dense , Bidirectional , ConvLSTM2D , concatenate , Reshape , CuDNNGRU , subtract , SpatialDropout2D

from keras import backend
from keras.callbacks import Callback as Callback
from keras.backend import tensorflow_backend as K

import tensorflow as tf


################################################################################
dirlocal = './'
dataset = 'full' # 'sample' or 'full'
 
modelfile = 'model-' + str(stamp) + '.h5'

max_epochs = 64
es_patience = 100

if dataset == 'sample':
    max_epochs = 8
    es_patience = 1



In [ ]:

class NDSRobust(TransformerMixin):
    def __init__(self, **kwargs):
        self._scaler = RobustScaler(copy=True, **kwargs)
        self._orig_shape = None

    def fit(self, X, **kwargs):
        X = np.array(X)
        # Save the original shape to reshape the flattened X later
        # back to its original shape
        
        if len(X.shape) > 1:
            self._orig_shape = X.shape[1:]
        X = self._flatten(X)
        self._scaler.fit(X, **kwargs)
        return self

    def transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.transform(X, **kwargs)
        X = self._reshape(X)
        return X
    
    def inverse_transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.inverse_transform(X, **kwargs)
        X = self._reshape(X)
        return X
    
    def _flatten(self, X):
        # Reshape X to <= 2 dimensions
        if len(X.shape) > 2:
            n_dims = np.prod(self._orig_shape)
            X = X.reshape(-1, n_dims)
        return X

    def _reshape(self, X):
        # Reshape X back to it's original shape
        if len(X.shape) >= 2:
            X = X.reshape(-1, *self._orig_shape)
        return X


class NDSPCA(TransformerMixin):
    def __init__(self, **kwargs):
        self._scaler = PCA(copy = True, **kwargs)
        self._orig_shape = None

    def fit(self, X, **kwargs):
        X = np.array(X)
        # Save the original shape to reshape the flattened X later
        # back to its original shape
        
        if len(X.shape) > 1:
            self._orig_shape = X.shape[1:]
        X = self._flatten(X)
        self._scaler.fit(X, **kwargs)
        self.explained_variance_ratio_ = self._scaler.explained_variance_ratio_
        self.components_ =self._scaler.components_
        
        return self

    def transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.transform(X, **kwargs)
        
        return X
    
    def inverse_transform(self, X, **kwargs):
        X = np.array(X)
        X = self._flatten(X)
        X = self._scaler.inverse_transform(X, **kwargs)
        X = self._reshape(X)
        return X

    def _flatten(self, X):
        # Reshape X to <= 2 dimensions
        if len(X.shape) > 2:
            n_dims = np.prod(self._orig_shape)
            X = X.reshape(-1, n_dims)
        return X

    def _reshape(self, X):
        # Reshape X back to it's original shape
        if len(X.shape) >= 2:
            X = X.reshape(-1, *self._orig_shape)
        return X
    

def fit_y( y , components = 300 , FFT = False ):
    if FFT == True:
        y = np.stack([ np.fft.rfft2(y[i,:,:]) for i in range(y.shape[0])] )
        print(y.shape)
        y =  np.hstack( [ np.real(y) , np.imag(y)]  )
    print(y.shape)
    ndpca = NDSPCA(n_components=components)
    ndpca.fit(y)
    print('explained variance')
    print(np.sum(ndpca.explained_variance_ratio_))
    y = ndpca.transform(y)
    scaler0 = RobustScaler( )
    scaler0.fit(y)
    return scaler0, ndpca 

def transform_y( y, scaler,ndpca , FFT = False ):
    if FFT == True:
        y = np.stack([ np.fft.rfft2(y[i,:,:]) for i in range(y.shape[0])] )
        print(y.shape)
        y =  np.hstack( [ np.real(y) , np.imag(y)]  )
    y = ndpca.transform(y)
    print(y.shape)
    y = scaler0.transform(y)
    
    return y 

def inverse_transform_y( y , scaler, ndpca , FFT=False):
    
    y = scaler0.inverse_transform(y)
    y = ndpca.inverse_transform(y)

    if FFT == True:
        split = int(y.shape[1]/2)
        y = np.stack([ np.fft.irfft2(y[i,:split,:] + 1j*y[i,split:,:]) for i in range(y.shape[0]) ] )
                  
    return y
    

In [9]:
### todo, get MSAs, get pdb distmats, get pdb GO
import pandas as pd
uniclust = '/home/cactuskid13/mntpt/HHBLITsdb/uniclust30_2018_08/uniclust30_2018_08.tsv'
dfclust = pd.read_csv( uniclust , comment = '!' , sep = '\t' , header = None )
print(dfclust)

                    0           1
0          A0A1Q6BRU5  A0A1Q6BRU5
1          A0A0A9U2E6  A0A0A9U2E6
2          A0A1Y6B8R2  A0A1Y6B8R2
3          A0A1Y6B8R2  A0A2E8I3A4
4          A0A094PJX3  A0A094PJX3
5          A0A0P6A7Z7  A0A0P6A7Z7
6          A0A2E2XLC2  A0A2E2XLC2
7          A0A0L9UG40  A0A0L9UG40
8          A0A2W6D8L2  A0A2W6D8L2
9          A0A1R1PVC7  A0A1R1PVC7
10         A0A200R9F7  A0A200R9F7
11         A0A1Q5T2G7  A0A1Q5T2G7
12         A0A0J6YAJ9  A0A0J6YAJ9
13         A0A0J6YAJ9  A0A0J8RQ30
14         A0A0J6YAJ9  A0A0J8QT49
15         A0A0J6YAJ9      E9D7I5
16         A0A0J6YAJ9  A0A0J6FKZ9
17         A0A2B2HHJ2  A0A2B2HHJ2
18         A0A2B2HHJ2  A0A1X6Q0Z3
19         A0A2B2HHJ2      R8HZC0
20         A0A2B2HHJ2      R8PED3
21         A0A2B2HHJ2  A0A1V9W5F1
22         A0A2B2HHJ2  A0A2A7HR05
23         A0A2B2HHJ2  A0A2B9B3R7
24         A0A2B2HHJ2  A0A150BE34
25         A0A2B2HHJ2      K0FM31
26         A0A2B2HHJ2  A0A2T0EFU9
27         A0A2B2HHJ2  A0A1J9Y0R4
28         A0A

In [12]:
#pdb to uniprot map w uniclust cluster rep for each struct
#grab aln for clust 

pdbuni = './pdb_chain_uniprot.csv'
dfuni = pd.read_csv( pdbuni , low_memory = False , comment = '#') 
dfuni = dfuni.merge(dfclust , how= 'left' , right_on = 1 , left_on = 'SP_PRIMARY' )
print(dfuni)


         PDB CHAIN SP_PRIMARY  RES_BEG  RES_END PDB_BEG PDB_END  SP_BEG  \
0       101m     A     P02185        1      154       0     153       1   
1       102l     A     P00720        1       40       1      40       1   
2       102l     A     P00720       42      165      41    None      41   
3       102m     A     P02185        1      154       0     153       1   
4       103l     A     P00720        1       40       1    None       1   
5       103l     A     P00720       44      167      41    None      41   
6       103m     A     P02185        1      154       0     153       1   
7       104l     A     P00720        1       44       1      44       1   
8       104l     A     P00720       47      166      45    None      45   
9       104l     B     P00720        1       44       1      44       1   
10      104l     B     P00720       47      166      45    None      45   
11      104m     A     P02185        1      153       1     153       2   
12      105m     A     P0

In [ ]:
#use column 0 to retreive aln for pdb



In [15]:
#merge with GO terms
pdbgo = './pdb_chain_go.csv'
dfgo = pd.read_csv( pdbgo , error_bad_lines=False , warn_bad_lines= False ,comment = '#')
print(dfgo)


           PDB CHAIN SP_PRIMARY                WITH_STRING EVIDENCE  \
0         101m     A       IPRO         InterPro:IPR000971      IEA   
1         101m     A       IPRO         InterPro:IPR002335      IEA   
2         101m     A       IPRO         InterPro:IPR002335      IEA   
3         101m     A       IPRO         InterPro:IPR002335      IEA   
4         101m     A       IPRO         InterPro:IPR012292      IEA   
5         101m     A       IPRO         InterPro:IPR012292      IEA   
6         101m     A     P02185       UniProtKB-KW:KW-0479      IEA   
7         101m     A     P02185       UniProtKB-KW:KW-0561      IEA   
8         101m     A     P02185       UniProtKB-KW:KW-0561      IEA   
9         102l     A       IPRO         InterPro:IPR001165      IEA   
10        102l     A       IPRO         InterPro:IPR001165      IEA   
11        102l     A       IPRO         InterPro:IPR002196      IEA   
12        102l     A       IPRO         InterPro:IPR002196      IEA   
13    

In [ ]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction= 0.90
K.set_session(tf.Session(config=config))

#systematicall probe net arch
#insert bottleneck in dense

#first part of the network, stacked GRU
LSTMoutdim_max = 500
LSTMoutdim_min = 500

LSTMlayers = 2

Densegrulayers = 2
Densegruoutdim_min = 300
Densegruoutdim_max = 300

retrain = False
print(XTRAINrows.shape)
print(XVALIDrows.shape)

def bottleneck( minn, maxn, layers ):
    split = int(layers/2)
    slope = (minn - maxn) / split
    units =[maxn]    
    last = maxn
    for k in range(split):
        last += slope 
        units.append(int(last))
    slope = -slope
    for k in range(split):
        last += slope 
        units.append(int(last))
    return units

lstmbottle = bottleneck(LSTMoutdim_min , LSTMoutdim_max  , LSTMlayers )
bottle = bottleneck(Densegruoutdim_min , Densegruoutdim_max  ,Densegrulayers )
print(lstmbottle)


In [ ]:
retrain = True
if retrain == False:
    inputrnn = Input(name='Seqin', shape=( XTRAINrows.shape[1] ,XTRAINrows.shape[2] ) )
    layer = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones')
    x = layer(inputrnn)
    print(x)
    
    
    for b in lstmbottle:
            #layer = CuDNNGRU(LSTMoutdim ,  name='gru_'+str(n),
            #return_sequences=True, return_state=False, go_backwards=False, stateful=False )
            
            layer = CuDNNLSTM(b , kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal',
                    bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, 
                    recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
                    recurrent_constraint=None, bias_constraint=None, return_sequences=True, return_state=False, stateful=False)
            
            layer = Bidirectional(layer, merge_mode='concat', weights=None)
            x = layer(x)
            
            layer = Dropout(.5 , noise_shape=None, seed=None)
            x = layer(x)
            
            layer = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones')
            x = layer(x)

    #layer = CuDNNGRU(LSTMoutdim ,  name='gru_'+str(n+1),
    #return_sequences=True, return_state=False, go_backwards=False, stateful=False )
    
    #todo... regularizers 
    layer = CuDNNLSTM( LSTMoutdim_max , kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
                              bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None, 
                              recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, 
                              recurrent_constraint=None, bias_constraint=None, return_sequences=True, return_state=False, stateful=False)
    x = layer(x)
    layer = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones')
    x = layer(x)

    x =  Flatten()(x)
    layer = Dropout(.5 , noise_shape=None, seed=None)
    x = layer(x)
    
    layer = Dense( YTRAINpca.shape[1] , activation='linear'  )
    x = layer(x)
    
    
    layer = Dense( YTRAINpca.shape[1] , activation='linear'  )
    xpre= layer(x)
    
    
    #final transform to img
    layer = Dense( YTRAIN.shape[2] * YTRAIN.shape[1] , activation='linear'  )
    x = layer(xpre)
    
    layer = BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones')
    x = layer(x)
    
    layer = Reshape( ( YTRAIN.shape[1], YTRAIN.shape[2] ))
    xfinal = layer(x)
    
    
    with tf.device('/cpu:0'):
        model = Model(inputs = [inputrnn] , outputs = [xpre,xfinal] )
    o = keras.optimizers.Adadelta(lr=.0001, rho=0.95)
    model.compile( optimizer=o, loss= 'mean_absolute_percentage_error' , metrics=['mae'] )
else:
    model = print('Load the model..')
    modelfile = 'model-12_01_2019_12_12_34_169126.h5'
    model = load_model(modelfile, custom_objects= { 'tf' : tf  } )
    o = keras.optimizers.Adadelta(lr=.025, rho=0.95)
    model.compile( optimizer=o, loss= 'mean_absolute_error' , metrics=['mae'])

max_len = 3000
mc = ModelCheckpoint(modelfile, monitor = 'val_loss', mode = 'min', verbose = 1, save_best_only = True)
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 2, patience = es_patience)
history = model.fit( { 'Seqin': XTRAINrows } , [YTRAINpca,YTRAIN] , batch_size = None , verbose = 2 , epochs = 100000 ,  validation_data=([XVALIDrows], [YVALIDpca,YVALID] ), callbacks=[es, mc])